In [ ]:
# Import some basic libraries and functions for this tutorial.
import numpy as np
import os

from pydrake.common import FindResourceOrThrow, temp_directory
from pydrake.geometry import (
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Role,
    StartMeshcat,
)
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.meshcat import JointSliders
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, MultibodyPlant
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseDynamicsController, PiecewisePose, Rgba, LeafSystem, JacobianWrtVariable, TrajectorySource, ModelInstanceIndex


In [ ]:
meshcat = StartMeshcat()

In [ ]:
filename = FindResourceOrThrow("drake/manipulation/models/ur3e/ur3e_cylinders_collision.urdf")


In [ ]:
meshcat.Delete()
meshcat.DeleteAddedControls()
builder = DiagramBuilder()

# Note: the time_step here is chosen arbitrarily.
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

# Load the file into the plant/scene_graph.
parser = Parser(plant, scene_graph)  
parser.AddModelFromFile(filename)

plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("ur_base_link"))

plant.Finalize()

# Add two visualizers, one to publish the "visual" geometry, and one to
# publish the "collision" geometry.
visual = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat, MeshcatVisualizerParams(role=Role.kPerception, prefix="visual")
)
collision = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat, MeshcatVisualizerParams(role=Role.kProximity, prefix="collision")
)
# Disable the collision geometry at the start; it can be enabled by the
# checkbox in the meshcat controls.
meshcat.SetProperty("collision", "visible", False)

# Set the timeout to a small number in test mode. Otherwise, JointSliders
# will run until "Stop JointSliders" button is clicked.
default_interactive_timeout = 1.0 if "TEST_SRCDIR" in os.environ else None
sliders = builder.AddSystem(JointSliders(meshcat, plant))
diagram = builder.Build()
sliders.Run(diagram, default_interactive_timeout)


In [ ]:
meshcat.Delete()
meshcat.DeleteAddedControls()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

model = Parser(plant).AddModelFromFile(filename)
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("ur_base_link"))
plant.Finalize()


Kp = np.full(6, 100)
Ki = 2 * np.sqrt(Kp)
Kd = np.full(6, 20)
controller = builder.AddSystem(InverseDynamicsController(plant, Kp, Ki, Kd, False))
controller.set_name("controller");
builder.Connect(plant.get_state_output_port(model),
                controller.get_input_port_estimated_state())
builder.Connect(controller.get_output_port_control(),
                plant.get_actuation_input_port())


diagram = builder.Build()

context = diagram.CreateDefaultContext()
# plant_context = plant.GetMyMutableContextFromRoot(context)


# plant.SetPositions(plant_context, [0, -np.pi/2, 0, -np.pi/2 + np.pi /64, 0, 0])
# plant.get_actuation_input_port().FixValue(plant_context, np.zeros(6))

plant_context = plant.GetMyMutableContextFromRoot(context)
q0 = np.array([0, -np.pi/2, 0, -np.pi/2 + np.pi /64, 0, 0])
x0 = np.hstack((q0, 0*q0))
plant.SetPositions(plant_context, q0)

x0[0] += np.pi / 2
controller.GetInputPort('desired_state').FixValue(controller.GetMyMutableContextFromRoot(context), x0)

simulator = Simulator(diagram, context)

visualizer.StartRecording()
simulator.AdvanceTo(10.0)
visualizer.StopRecording()
visualizer.PublishRecording()

In [ ]:
from airo_drake.visualization import AddMeshcatTriad

def visualize_link(name):
    eef = plant.GetBodyByName(name, model)
    X_Ginitial = plant.EvalBodyPoseInWorld(plant_context, eef)
    print(X_Ginitial)

    AddMeshcatTriad(meshcat, name, X_PT=X_Ginitial)

visualize_link("ur_ee_link")
visualize_link("ur_tool0")

In [ ]:
for i in range(plant.num_model_instances()):
    model_instance = ModelInstanceIndex(i)
    model_instance_name = plant.GetModelInstanceName(model_instance)
    print(model_instance_name)